In [ ]:
from pysat.formula import IDPool
from pysat.solvers import Solver

# Define grid size and numbers
grid_size = 9
numbers = range(1, grid_size + 1)

# Initialize ID pool for variables mapping
vpool = IDPool()

def varnum(i, j, k):
    """Encode cell positions and number into a unique variable."""
    return vpool.id(f"{i}_{j}_{k}")

def at_most_one_in_cell(solver):
    """Ensure at most one number per cell."""
    for i in numbers:
        for j in numbers:
            for k1 in numbers:
                for k2 in numbers:
                    if k1 < k2:
                        solver.add_clause([-varnum(i, j, k1), -varnum(i, j, k2)])

def row_constraints(solver):
    """Ensure each number appears exactly once per row."""
    for k in numbers:
        for i in numbers:
            solver.add_clause([varnum(i, j, k) for j in numbers])

def column_constraints(solver):
    """Ensure each number appears exactly once per column."""
    for k in numbers:
        for j in numbers:
            solver.add_clause([varnum(i, j, k) for i in numbers])

def subgrid_constraints(solver):
    """Ensure each number appears exactly once per subgrid."""
    for k in numbers:
        for bi in range(0, grid_size, 3):
            for bj in range(0, grid_size, 3):
                solver.add_clause([varnum(bi + i, bj + j, k)
                                   for i in range(3) for j in range(3)])

def solve_sudoku(puzzle):
    """Solve Sudoku puzzle."""
    solver = Solver()

    # Apply all constraints to the solver
    at_most_one_in_cell(solver)
    row_constraints(solver)
    column_constraints(solver)
    subgrid_constraints(solver)

    # Encode puzzle clues
    for i in numbers:
        for j in numbers:
            if puzzle[i-1][j-1] != 0:  # Assuming puzzle is a 9x9 list with 0 as empty cells
                solver.add_clause([varnum(i, j, puzzle[i-1][j-1])])

    # Solve the puzzle
    is_solvable = solver.solve()
    if not is_solvable:
        return None

    # Extract solution from the model
    solution = [[0 for _ in range(grid_size)] for _ in range(grid_size)]
    for i in numbers:
        for j in numbers:
            for k in numbers:
                if solver.get_model()[varnum(i, j, k) - 1] > 0:
                    solution[i-1][j-1] = k
    return solution

# Example puzzle (0 represents empty cells)
puzzle = [
    [5, 3, 0, 0, 7, 0, 0, 0, 0],
    [6, 0, 0, 1, 9, 5, 0, 0, 0],
    [0, 9, 8, 0, 0, 0, 0, 6, 0],
    [8, 0, 0, 0, 6, 0, 0, 0, 3],
    [4, 0, 0, 8, 0, 3, 0, 0, 1],
    [7, 0, 0, 0, 2, 0, 0, 0, 6],
    [0, 6, 0, 0, 0, 0, 2, 8, 0],
    [0, 0, 0, 4, 1, 9, 0, 0, 5],
    [0, 0, 0, 0, 8, 0, 0, 7, 9]
]

# Solve the puzzle
solution = solve_sudoku(puzzle)
if solution:
    for row in solution:
        print(row)
else:
    print("No solution found.")
